In [4]:
import os, time, imageio, itertools, pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize
import imageio
from utils import *

# 함수정의


In [2]:
def center_crop(x,crop_w=108,crop_h=108,resize_h=64,resize_w=64):
    h,w = x.shape[0], x.shape[1]
    j = int(round((h-crop_h)/2.))
    i = int(round((w-crop_w)/2.))
    return resize(x[j:j+crop_h,i:i+crop_w],(resize_h,resize_w),mode='reflect')
    

In [3]:
class batch_norm(object):
    def __init__(self, epsilon=1e-5, momentum = 0.9, name="batch_norm"):
        with tf.variable_scope(name):
            self.epsilon  = epsilon
            self.momentum = momentum
            self.name = name

    def __call__(self, x, train=True):
        return tf.contrib.layers.batch_norm(x,
                      decay=self.momentum, 
                      updates_collections=None,
                      epsilon=self.epsilon,
                      scale=True,
                      is_training=train,
                      scope=self.name)

def leakyReLU(x,leak=0.2,name='leakyReLU'):
    return tf.maximum(x,leak*x)

def conv2d(input_, output_dim, 
       k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
       name="conv2d"):
        with tf.variable_scope(name):
            w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              initializer=tf.truncated_normal_initializer(stddev=stddev))
            conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')

            biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
            conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return conv

def conv_out_size_same(size, stride):
    return int(math.ceil(float(size) / float(stride)))

def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
          initializer=tf.constant_initializer(bias_start))
    if with_w:
        return tf.matmul(input_, matrix) + bias, matrix, bias
    else:
        return tf.matmul(input_, matrix) + bias

def deconv2d(input_, output_shape,
        k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
        name="deconv2d", with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                  initializer=tf.random_normal_initializer(stddev=stddev))
    
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                    strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                    strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv
 


# DCGAN정의

In [4]:
class DCGAN(object):
    def __init__(self,sess,input_height=64,input_width=64,batch_size=64,
                 output_height=64,output_width=64,z_dim=100,gf_dim=64,df_dim=64,
                 dfc_dim=1024,gfc_dim=1024,color_dim=3):
        '''
        
        Args:
            sess : Tensorflow session
            batch_size : The size of batch.
            z_dim = Dimension of dim for z
            gf_dim = Dimension of generator filters in first conv layer
            df_dim = Dimension of discriminator filters in frist conv layer
            gfc_dim = Dimension of generator units for fully connected layer
            dfc_dim = Dimension of discriminaotr units for fully connected layer
            color_dim = image color. For grayscale input, set to 1.
        '''
        self.sess = sess
        self.batch_size = batch_size
        self.z_dim=z_dim
        self.input_height = input_height
        self.input_width = input_width
        self.output_height = output_height
        self.output_width = output_width
        
        self.gf_dim = gf_dim
        self.df_dim = df_dim
        
        self.gfc_dim = gfc_dim
        self.dfc_dim = dfc_dim
        self.color_dim=color_dim
        # batch normalization : deals with poor initialization helps gradient flow
        self.d_bn1 = batch_norm(name = 'd_bn1') 
        self.d_bn2 = batch_norm(name = 'd_bn2')
        self.d_bn3 = batch_norm(name = 'd_bn3')
        
        self.g_bn0 = batch_norm(name = 'g_bn0')
        self.g_bn1 = batch_norm(name = 'g_bn1')
        self.g_bn2 = batch_norm(name = 'g_bn2')
        self.g_bn3 = batch_norm(name = 'g_bn3')
    
    def build_model(self):    
        
        self.input_x = tf.placeholder(
        tf.float32,[self.batch_size,self.input_height,self.input_width,self.color_dim],name='real_images')
        
        self.z = tf.placeholder(
        tf.float32,[None,self.z_dim],name='f')
        
        self.G = self.generator(self.z)
        self.D, self.D_logits = self.discriminator(self.input_x,reuse=False)
        self.D_, self.D_logits_ = self.discriminator(self.G,reuse=True)
        
        self.d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_logits,labels = tf.ones_like(self.D)))
        
        self.d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_logits_,labels = tf.zeros_like(self.D)))
        
        self.g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_logits_,labels = tf.ones_like(self.D)))

        self.d_loss = self.d_loss_real + self.d_loss_fake
        
        t_vars = tf.trainable_variables()
        
        self.d_vars = [var for var in t_vars if 'd_' in var.name]
        self.g_vars = [var for var in t_vars if 'g_' in var.name]
        
    
    def train(self,data,epoches,learning_rate,beta1):
        self.d_optim = tf.train.AdamOptimizer(learning_rate,beta1 = beta1).minimize(self.d_loss,var_list=self.d_vars)
        self.g_optim = tf.train.AdamOptimizer(learning_rate,beta1 = beta1).minimize(self.g_loss,var_list=self.g_vars)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
        self.data = data
        sample_z = np.random.uniform(-1,1,size=(self.batch_size,self.z_dim))
        
        # results save folder
        if not os.path.isdir("result"):
            os.mkdir('result')
        
        iter_per_epoch = len(self.data)//self.batch_size
        counter=1
        self.train_hist={}
        self.train_hist['D_losses'] = []
        self.train_hist['G_losses'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['epoch'] = []
        
        
        for epoch in range(epoches):
            start_time=time.time()
            for idx in range(iter_per_epoch):
                batch_images = self.data[idx*self.batch_size:(idx+1)*self.batch_size]
                
                batch_z = np.random.uniform(-1,1,[self.batch_size,self.z_dim]).astype(np.float32)

                # update D network
                _,d_losses= self.sess.run([self.d_optim,self.d_loss],feed_dict={self.input_x:batch_images,self.z:batch_z})
        
                # update G network
                _= self.sess.run(self.g_optim,feed_dict={self.z:batch_z})
                # twice update G network
                _,g_losses = self.sess.run([self.g_optim,self.g_loss],feed_dict={self.z:batch_z})
                
                errD_fake = sess.run(self.d_loss_fake,feed_dict={
                    self.z : batch_z
                })
                errD_real = sess.run(self.d_loss_real,feed_dict={
                    self.input_x : batch_images
                })
                errG = sess.run(self.g_loss,feed_dict={
                    self.z : batch_z
                })
            
                counter += 1
            samples = sess.run(self.G,feed_dict={self.z:sample_z})
                    
            save_images(samples, image_manifold_size(samples.shape[0]),
                    './{}/train_{:02d}_{:04d}.png'.format('./result', epoch, idx))
                                       
            print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
                              % (epoch, idx,  iter_per_epoch,
                                time.time() - start_time, errD_fake+errD_real, errG))
            self.train_hist['D_losses'].append(d_losses)
            self.train_hist['G_losses'].append(g_losses)
            self.train_hist['per_epoch_ptimes'].append(time.time()-start_time)
            self.train_hist['epoch'].append(epoch)
        
        
    def discriminator(self,image,reuse=False):
        with tf.variable_scope('discriminator') as scope:
            # 나중에 fake image도 들어가기 때문에 
            if reuse:
                scope.reuse_variables()
                
            h0 = leakyReLU(conv2d(image,self.df_dim,name='d_h0_conv'))
            h1 = leakyReLU(self.d_bn1(conv2d(h0,self.df_dim*2,name='d_h1_conv')))
            h2 = leakyReLU(self.d_bn2(conv2d(h1,self.df_dim*4,name='d_h2_conv')))
            h3 = leakyReLU(self.d_bn3(conv2d(h2,self.df_dim*8,name='d_h3_conv')))
            h4 = linear(tf.reshape(h3,[self.batch_size,-1]),1,'d_h4_lin')
        
        return tf.nn.sigmoid(h4), h4

    def generator(self,z):
        with tf.variable_scope('generator'):
            
            s_h, s_w = self.output_height, self.output_width
            s_h2, s_w2 = conv_out_size_same(s_h,2),conv_out_size_same(s_w,2)
            s_h4, s_w4 = conv_out_size_same(s_h2,2),conv_out_size_same(s_w2,2)
            s_h8, s_w8 = conv_out_size_same(s_h4,2),conv_out_size_same(s_w4,2)
            s_h16, s_w16 = conv_out_size_same(s_h8,2),conv_out_size_same(s_w8,2)
            
            # project 'z' and reshape
            self.z_, self.h0_w,self.h0_b = linear(
            z, self.gf_dim*8*s_h16*s_w16,'g_h0_lin',with_w=True)
            
            self.h0 = tf.reshape(
            self.z_,[-1,s_h16,s_w16,self.gf_dim*8])
            h0 = tf.nn.relu(self.g_bn0(self.h0))
            
            self.h1, self.h1_w, self.h1_b = deconv2d(
            h0,[self.batch_size,s_h8,s_w8,self.gf_dim*4],name='g_h1',with_w=True)
            h1 = tf.nn.relu(self.g_bn1(self.h1))
            
            self.h2, self.h2_w, self.h2_b = deconv2d(
            h1,[self.batch_size,s_h4,s_w4,self.gf_dim*2],name='g_h2',with_w=True)
            h2 = tf.nn.relu(self.g_bn2(self.h2))
            
            self.h3, self.h3_w, self.h3_b = deconv2d(
            h2,[self.batch_size,s_h2,s_w2,self.gf_dim*1],name='g_h3',with_w=True)
            h3 = tf.nn.relu(self.g_bn3(self.h3))
            
            self.h4, self.h4_w, self.h4_b = deconv2d(
            h3,[self.batch_size,s_h,s_w,self.color_dim],name='g_h4',with_w=True)
            
            return tf.nn.tanh(self.h4)
        

# 데이터 불러오기

In [3]:
# setting a directory
data_dir = '/home/snu/study/GAN/DCGAN/DCGAN-tensorflow/data/celebA'

# data load
train_load = io.ImageCollection(data_dir + '/*.jpg')



In [ ]:
data_x = train_load[0]

In [ ]:
data_x = np.array([center_crop(img) for img in train_load[0:100000]])

# 하이퍼 파라미터

In [ ]:
learning_rate = 0.0002
epoch=400

# 실행

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    dcgan = DCGAN(sess=sess)

In [ ]:
dcgan.build_model()

In [ ]:
dcgan.train(data_x,learning_rate=0.0002,epoches=epoch,beta1=0.5)

# 애니메이션 만들기

In [14]:
# setting a directory
result_dir = './celebA_result'

# data load
result_load = io.ImageCollection(result_dir + '/*.png')

images=[]
# image들을 gif파일로 저장
for result in result_load:
        images.append(result)

imageio.mimsave('generation_animation.gif',images,fps=5)
